In [1]:

import torch

from lightning.pytorch.callbacks import ModelCheckpoint, RichModelSummary

from rl4co.envs import SSPkoptEnv, ATSPkoptEnv, TSPkoptEnv
from rl4co.models import NeuOptPolicy, NeuOpt
from rl4co.models.nn.env_embeddings.edge import ATSPEdgeEmbedding
from rl4co.utils.trainer import RL4COTrainer


In [2]:
import torch.nn as nn
from rl4co.envs.routing.atsp.generator import ATSPCoordGenerator

def batch_svd_embed(cost_matrix, k=2):
    """
    For each sample in [B,N,N], do an SVD on the N×N cost matrix
    and use the top-k singular vectors to embed each row in k-D.
    Returns [B, N, k].
    """
    B, N, _ = cost_matrix.shape
    coords = []
    for b in range(B):
        C = cost_matrix[b]                # [N,N]
        # SVD: C = U @ diag(S) @ V^T
        U, S, Vh = torch.linalg.svd(C)    # U: [N,N], S: [N], Vh: [N,N]
        # Row embeddings: multiply U[:, :k] by sqrt of singular values
        X = U[:, :k] * S[:k].sqrt()       # [N, k]
        coords.append(X)
    return torch.stack(coords, dim=0)      # [B, N, k]

class CustomizeATSPInitEmbedding(nn.Module):
    def __init__(self, embed_dim, num_loc, linear_bias=True):
        super(CustomizeATSPInitEmbedding, self).__init__()
        node_dim = 2
        self.init_embed = nn.Sequential(
            # nn.LayerNorm(node_dim),
            nn.Linear(node_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim),
        )


    def forward(self, td):
        with torch.no_grad():
            coords = batch_svd_embed(td["cost_matrix"], k=2)  # [B,N,2]
        out = self.init_embed(coords)
        return out

num_loc = 10  # Number of strings
fixed_len = 15
embed_dim = 128  # Dimension of the embedding space

env = SSPkoptEnv(generator_params=dict(num_loc=num_loc, fixed_len=fixed_len, init_sol_type="random"), k_max=4)
model = NeuOpt(
        env,
        batch_size=128,
        train_data_size=1000,
        val_data_size=100,
        test_data_size=100,
        n_step=5,
        T_train=200,
        T_test=1000,
        CL_best=True,
        policy_kwargs=dict(
            embed_dim=embed_dim,
            init_embedding=CustomizeATSPInitEmbedding(num_loc=num_loc,embed_dim=embed_dim),
        ),
    )

/home/shora/Research/rl4co/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'critic' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['critic'])`.


In [3]:
checkpoint_callback = ModelCheckpoint(  dirpath="checkpoints_ssp", # save to checkpoints/
                                        filename="atsp_mds{epoch:03d}",  # save as epoch_XXX.ckpt
                                        save_top_k=1, # save only the best model
                                        save_last=True, # save the last model
                                        monitor="val/cost_bsf", # monitor validation reward
                                        mode="min") # maximize validation reward


rich_model_summary = RichModelSummary(max_depth=3)

callbacks = [checkpoint_callback, rich_model_summary]

In [4]:
from lightning.pytorch.loggers import WandbLogger
logger = WandbLogger(project="rl4co", name="atsp_mds", log_model=True, save_dir="wandb_logs")

#  logger = None


In [5]:
trainer = RL4COTrainer(
    max_epochs=20,
    gradient_clip_val=0.05,
    devices=1,
    accelerator="gpu",
    logger=logger,
    callbacks=callbacks,
)

Using 16bit Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer.test(model)

wandb: Currently logged in as: shoraaa (shoraaa-vnu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


val_file not set. Generating dataset instead
test_file not set. Generating dataset instead
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/shora/Research/rl4co/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# Fit and test the model
trainer.fit(model)
trainer.test(model)

Overriding gradient_clip_val to None for 'automatic_optimization=False' models
/home/shora/Research/rl4co/.venv/lib/python3.13/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:658: Checkpoint directory /home/shora/Research/rl4co/train/checkpoints_tsp exists and is not empty.
/home/shora/Research/rl4co/.venv/lib/python3.13/site-packages/sklearn/manifold/_mds.py:677: FutureWarning: The default value of `n_init` will change from 4 to 1 in 1.9.
  warnings.warn(
/home/shora/Research/rl4co/.venv/lib/python3.13/site-packages/sklearn/manifold/_mds.py:677: FutureWarning: The default value of `n_init` will change from 4 to 1 in 1.9.
  warnings.warn(
/home/shora/Research/rl4co/.venv/lib/python3.13/site-packages/sklearn/manifold/_mds.py:677: FutureWarning: The default value of `n_init` will change from 4 to 1 in 1.9.
  warnings.warn(
/home/shora/Research/rl4co/.venv/lib/python3.13/site-packages/sklearn/manifold/_mds.py:677: FutureWarning: The default value of `n_init` will change from

┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃    ┃ Name                            ┃ Type                       ┃ Params ┃ Mode  ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0  │ env                             │ ATSPkoptEnv                │      0 │ train │
│ 1  │ policy                          │ NeuOptPolicy               │  691 K │ train │
│ 2  │ policy.encoder                  │ N2SEncoder                 │  344 K │ train │
│ 3  │ policy.encoder.init_embedding   │ CustomizeATSPInitEmbedding │ 16.9 K │ train │
│ 4  │ policy.encoder.pos_embedding    │ CyclicPositionalEmbedding  │      0 │ train │
│ 5  │ policy.encoder.pos_net          │ MultiHeadCompat            │ 32.8 K │ train │
│ 6  │ policy.encoder.net              │ AdaptiveSequential         │  295 K │ train │
│ 7  │ policy.decoder                  │ RDSDecoder                 │  329 K │ train │
│ 8  │ policy.decoder.linear_K1        │ Linear                     │ 16.4 K │ train │
│ 9  │ policy.decoder.linear_K2        │ Linear                     │ 16.4 K │ train │
│ 10 │ policy.decoder.linear_K3        │ Linear                     │ 16.4 K │ train │
│ 11 │ policy.decoder.linear_K4        │ Linear                     │ 16.4 K │ train │
│ 12 │ policy.decoder.linear_Q1        │ Linear                     │ 16.4 K │ train │
│ 13 │ policy.decoder.linear_Q2        │ Linear                     │ 16.4 K │ train │
│ 14 │ policy.decoder.linear_Q3        │ Linear                     │ 16.4 K │ train │
│ 15 │ policy.decoder.linear_Q4        │ Linear                     │ 16.4 K │ train │
│ 16 │ policy.decoder.rnn1             │ GRUCell                    │ 99.1 K │ train │
│ 17 │ policy.decoder.rnn2             │ GRUCell                    │ 99.1 K │ train │
│ 18 │ policy.init_hidden_W            │ Linear                     │ 16.5 K │ train │
│ 19 │ critic                          │ CriticNetwork              │  140 K │ train │
│ 20 │ critic.encoder                  │ MultiHeadAttentionLayer    │ 98.6 K │ train │
│ 21 │ critic.encoder.0                │ SkipConnection             │ 65.5 K │ train │
│ 22 │ critic.encoder.1                │ Normalization              │      0 │ train │
│ 23 │ critic.encoder.2                │ SkipConnection             │ 33.0 K │ train │
│ 24 │ critic.encoder.3                │ Normalization              │      0 │ train │
│ 25 │ critic.value_head               │ CriticDecoder              │ 41.5 K │ train │
│ 26 │ critic.value_head.project_graph │ Linear                     │  8.3 K │ train │
│ 27 │ critic.value_head.project_node  │ Linear                     │  8.3 K │ train │
│ 28 │ critic.value_head.MLP           │ MLP                        │ 25.0 K │ train │
└────┴─────────────────────────────────┴────────────────────────────┴────────┴───────┘

Trainable params: 831 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 831 K                                                                                                
Total estimated model params size (MB): 3                                                                          
Modules in train mode: 94                                                                                          
Modules in eval mode: 0

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/shora/Research/rl4co/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/shora/Research/rl4co/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/shora/Research/rl4co/.venv/lib/python3.13/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]